In [7]:
#!pip install fastbook
#!pip install bert_score
#!pip install transformers
#! pip install sentencepiece
#!pip install ohmeow-blurr

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from fastai.text.all import *
from fastai import *
from blurr import *
from transformers import *
import transformers
from blurr.text.data.all import *
from blurr.text.modeling.all import *
import bert_score
from bert_score import score

import re
import numpy as np
import pandas as pd
import gzip
import io
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from fastbook import load_learner
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [6]:
train = pd.read_csv('articles2.csv')
#train2 = pd.read_csv('articles2.csv')
#train3 = pd.read_csv('articles3.csv')
#train.head()

In [7]:
train.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,53293,73471,Patriots Day Is Best When It Digs Past the Heroism,Atlantic,David Sims,2017-01-11,2017.0,1.0,NaN,"Patriots Day, Peter Berg’s new thriller that recreates the 2013 Boston Marathon bombing and the ensuing manhunt that followed it, is a surprisingly oblique, morally ambiguous movie from a typically straightforward filmmaker. Patriots Day takes an unexpectedly cynical view of the chaos, rash and bureaucratic infighting that followed the bombing — the question is whether that was Berg’s intended message. For its grim running time, the movie celebrates the men on the ground who helped bring the bombers to justice, but it’s the glimpses of something more complicated than jingoism that rea..."
1,53294,73472,A Break in the Search for the Origin of Complex Life,Atlantic,Ed Yong,2017-01-11,2017.0,1.0,NaN,"In Norse mythology, humans and our world were created by a pantheon of gods who lived in the realm of Asgard. As it turns out, these stories have a grain of truth to them. Thanks to a team of scientists led by Thijs Ettema, Asgard is now also the name of a large clan of microbes. Its members, which are named after Norse gods like Odin, Thor, Loki, and Heimdall, are found all over the world. Many of them are rare and no one has actually seen them under a microscope. But thanks to their DNA, we know they exist. And we know that they are singularly important to us, because they may well be th..."
2,53295,73474,Obama’s Ingenious Mention of Atticus Finch,Atlantic,Spencer Kornhaber,2017-01-11,2017.0,1.0,NaN,"“If our democracy is to work in this increasingly diverse nation,” Barack Obama said in his farewell address last night, “each one of us must try to heed the advice of one of the great characters in American fiction, Atticus Finch. ” He then quoted Finch: “You never really understand a person until you consider things from his point of view . .. until you climb into his skin and walk around in it.” Chelsea Clinton’s Apt Wrinkle in Time In the moment, it occurred to some viewers that Finch also provided quotes on diversity. Like, “Have you ever considered that you can’t have a set of ba..."
3,53296,73475,"Donald Trump Meets, and Assails, the Press",Atlantic,David A. Graham,2017-01-11,2017.0,1.0,NaN,"Updated on January 11 at 5:05 p. m. In his first press conference since July 2016, Donald Trump took only a few questions but made news on several fronts, saying he accepted the conclusion that Russia conducted hacks on top Democrats, bashing the press, and refusing once again to release his tax returns. Trump also refused to answer questions about whether any of his aides had been in contact with Russian officials, though he later said they had not as he departed the press conference. During the press conference, Trump announced a plan he said would answer concerns about conflicts of in..."
4,53297,73476,Trump: ’I Think’ Hacking Was Russian,Atlantic,Kaveh Waddell,2017-01-11,2017.0,1.0,NaN,"Updated at 12:25 p. m. After months of equivocating on the origin of cyberattacks that targeted Democrats before the election, Donald Trump said Wednesday that he thinks Russia was behind the intrusions. “As for hacking, I think it was Russian,” Trump said at a press conference in New York. “But I think we also get hacked by other countries and other people. ” Later, he emphasized his skepticism. “It could’ve been others also,” he said. At a meeting on Friday, the was briefed on classified intelligence by the heads of the NSA, FBI, and CIA, as well as the Director of National Intellige..."


In [3]:
#train = pd.concat([train1, train2, train3])

In [8]:
len(train)

49999

In [9]:
def proprocessing(sample): #preprocessing
    sample = re.sub(r"http\S+", "", sample) #url
    sample = re.sub('\n','', sample) #/n
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'  #special characters
    sample = re.sub(pat, '', sample)
    return sample

In [10]:
proprocessing(train['content'][16])

'The Trump administration could bring to the fore some education terms that have been largely overlooked in recent years as it moves to dramatically change the way students learn in the United States. And officials and advocacy groups will throw around other words that are commonly heard but not always thoroughly understood. As Congress prepares to decide whether   Donald Trumps pick for education secretary, Betsy DeVos, is fit for the job, its worth reviewing what some of the words and phrases that will be floating around in the coming days and months actually mean: School choice: Trump and DeVos have repeatedly said they are fans of school choice. Its a phrase that backers of charter schools see that definition below often use to avoid the more contentious charter wording. Broadly, proponents of school choice say they want students to be able to attend whatever school they and their families decide is best, whether its the traditional public school down the street or a charter across

In [11]:
train['cleantext']= train['content'].apply(lambda x:proprocessing(x))

In [15]:
#train.head()

In [12]:
train_texts = train[['cleantext','title']]

In [13]:
train_texts.head()

,cleantext,title
0,"Patriots Day, Peter Bergs new thriller that recreates the 2013 Boston Marathon bombing and the ensuing manhunt that followed it, is a surprisingly oblique, morally ambiguous movie from a typically straightforward filmmaker. Patriots Day takes an unexpectedly cynical view of the chaos, rash and bureaucratic infighting that followed the bombing the question is whether that was Bergs intended message. For its grim running time, the movie celebrates the men on the ground who helped bring the bombers to justice, but its the glimpses of something more complicated than jingoism that really ...",Patriots Day Is Best When It Digs Past the Heroism
1,"In Norse mythology, humans and our world were created by a pantheon of gods who lived in the realm of Asgard. As it turns out, these stories have a grain of truth to them. Thanks to a team of scientists led by Thijs Ettema, Asgard is now also the name of a large clan of microbes. Its members, which are named after Norse gods like Odin, Thor, Loki, and Heimdall, are found all over the world. Many of them are rare and no one has actually seen them under a microscope. But thanks to their DNA, we know they exist. And we know that they are singularly important to us, because they may well be th...",A Break in the Search for the Origin of Complex Life
2,"If our democracy is to work in this increasingly diverse nation, Barack Obama said in his farewell address last night, each one of us must try to heed the advice of one of the great characters in American fiction, Atticus Finch. He then quoted Finch: You never really understand a person until you consider things from his point of view . .. until you climb into his skin and walk around in it. Chelsea Clintons Apt Wrinkle in Time In the moment, it occurred to some viewers that Finch also provided quotes on diversity. Like, Have you ever considered that you cant have a set of backward pe...",Obama’s Ingenious Mention of Atticus Finch
3,"Updated on January 11 at 5:05 p. m. In his first press conference since July 2016, Donald Trump took only a few questions but made news on several fronts, saying he accepted the conclusion that Russia conducted hacks on top Democrats, bashing the press, and refusing once again to release his tax returns. Trump also refused to answer questions about whether any of his aides had been in contact with Russian officials, though he later said they had not as he departed the press conference. During the press conference, Trump announced a plan he said would answer concerns about conflicts of in...","Donald Trump Meets, and Assails, the Press"
4,"Updated at 12:25 p. m. After months of equivocating on the origin of cyberattacks that targeted Democrats before the election, Donald Trump said Wednesday that he thinks Russia was behind the intrusions. As for hacking, I think it was Russian, Trump said at a press conference in New York. But I think we also get hacked by other countries and other people. Later, he emphasized his skepticism. It couldve been others also, he said. At a meeting on Friday, the was briefed on classified intelligence by the heads of the NSA, FBI, and CIA, as well as the Director of National Intelligence. Th...",Trump: ’I Think’ Hacking Was Russian


In [14]:
train_texts, test_texts = train_test_split(train_texts, test_size=0.1)
len(train_texts)

44999

In [15]:
pretrained_model_name = "t5-base"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=T5ForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='cleantext',
    target_text_attr='title',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(train_texts)


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('cleantext'), get_y=ColReader('title'), splitter=RandomSplitter())
dls = dblock.dataloaders(preprocessed_train, bs = 2)

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the

In [16]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

In [17]:
#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

#Training
learn.fit_one_cycle(1, lr_max=3e-5, cbs=fit_cbs)

#Exporting model
learn.save('t5_headline_complete_v2.pkl')

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,nan,2.424182,0.291824,0.105794,0.261254,0.716704,0.705001,0.710286,1:30:08


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

Path('models/t5_headline_complete_v2.pkl.pth')

In [12]:
learner = learn.load('t5_headline_complete_v2.pkl')

In [13]:
learner.export('models/t5_production_headline_complete_v2.pkl')

In [14]:
inf_learn = load_learner('models/t5_production_headline_complete_v2.pkl')

In [30]:
content = ["Hello David, my name is George and I'm currently a MIDS candidate at UC Berkeley.I'm a big fan of your work and my team of data scientists and developers have created a product I think you’d be interested in. Do you know the hardest part about coming up with an elevator pitch? It’s trying to condense the most important parts into a concise number of words. That same difficulty can be found on Reddit, one of the fastest growing social media sites in the world. We’ve analyzed reddit data over a 10-year period and found that nearly 20% of all posts and articles had ineffective or erroneous titles. So, what’s the solution? Our product is a web-based app that takes a user’s post and generates a title that has been evaluated to accurately summarize the content. We're still in the early stages of development and would appreciate any feedback, would you be interested in meeting with my team to talk more through our product?"]
#content = ['TitleAi is focused on tackling the disconnect in relevance and interpretability between a Reddit title and its post, which is a pervasive issue among the various Reddit communities across the platform. ']

outputs = learn.blurr_generate(content, early_stopping=False, num_beams=4, num_return_sequences=4, \
                               min_length=5, max_length=20)
for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': ['We’re a web-based app that takes a user’s post and generate', 'We’re a web-based app that takes a user’s post and create', 'We’re a web-based app that takes a user’s post and produces', 'I’m a big fan of your work and I’m interested in your product']}



In [19]:
def f_rscore(reference, candidate): #assuming both are lists, Rouge
    rscores = []
    rscorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    for i in range(len(reference)):
        rscore = rscorer.score(reference[i], candidate[i])
        rscores.append(list(rscore.values())[0][2]) # fscore
    return rscores

def f_bscore(reference, candidate): #assuming both are lists, Bertscore
    P, R, F1 = score(candidate, reference, lang="en")
    return F1.tolist()

def make_predictions(test_texts):
    #list(outputs[0].values())
    predictions = []
    for i in tqdm(range(len(test_texts))):
        outputs = learn.blurr_generate(test_texts.iloc[i].cleantext, early_stopping=False, num_return_sequences=1, \
                                   min_length=5, max_length=20)
        predictions.append(list(outputs[0].values())[0])
    return predictions

def avg_rouge(test_texts, predictions):
    score_list = f_rscore(list(test_texts.title), predictions)
    return np.average(score_list)

def avg_bertscore(test_texts, predictions):
    score_list = f_bscore(list(test_texts.title), predictions)
    return np.average(score_list)

In [25]:
test_texts_sub = test_texts.sample(200)

In [26]:
predictions = make_predictions(test_texts_sub)

100%|█████████████████████████████████████████| 200/200 [00:39<00:00,  5.06it/s]


In [27]:
avg_rouge(test_texts_sub, predictions)

0.2523962855489208

In [23]:
avg_bertscore(test_texts_sub, predictions)

0.8835919350385666